In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
# Input directory for NMME forecast output
dirin = 'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME'
# Output directory for downloads
dirout = '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'#/NMME_SST/by_lead
# Model names
mods = [  'NCEP-CFSv2',]#'CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO','GFDL-SPEAR', 'NASA-GEOSS2S',
# Number of time steps to grab at each opendap call (to avoid download
# failures)
time_chunk = 10;

In [4]:
modi='CanSIPS-IC4'

In [5]:
str1='hindcast'

In [6]:
fin=f'{dirin}/{modi}/.{str1.upper()}/.MONTHLY/.sst/dods'
fin

'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/CanSIPS-IC4/.HINDCAST/.MONTHLY/.sst/dods'

In [7]:
fi=xr.open_dataset(fin,decode_times=False)

In [15]:
fi2=xr.open_dataset('https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.CanSIPS-IC4/.CanESM5/.HINDCAST/.MONTHLY/.sst/dods',
                     https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.CanSIPS-IC4/.CanESM5/.HINDCAST/.MONTHLY/.sst/dods
                   decode_times=False)

In [18]:
fi2.close()

In [19]:
fi.close()

In [10]:
fi.M

<xarray.DataArray 'M' (M: 40)> Size: 160B
array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40.],
      dtype=float32)
Coordinates:
  * M        (M) float32 160B 1.0 2.0 3.0 4.0 5.0 ... 36.0 37.0 38.0 39.0 40.0
Attributes:
    standard_name:  realization
    long_name:      Ensemble Member
    pointwidth:     1.0
    gridtype:       0
    units:          unitless

In [4]:
def _sourcepath(modi,str1):
    if modi=='COLA-RSMAS-CCSM4':
        fin=f'{dirin}/{modi}/.MONTHLY/.sst/dods'
    elif modi=='NCEP-CFSv2':
        fin=f'{dirin}/{modi}/HINDCAST/.PENTAD_SAMPLES_FULL/.sst/dods'
    elif modi in {'CanCM4i', 'GEM-NEMO','GFDL-SPEAR', 'NASA-GEOSS2S'}:
        fin=f'{dirin}/{modi}/.{str1.upper()}/.MONTHLY/.sst/dods'
    return fin
def dl(modi,str1='hindcast'):
    # options: 'hindcast' or 'forecast': handle models with separate forecast directories
    # Can models use a different order of dimensions from others (managed by xarray) 
    print(f'\nDownloading {modi} {str1}s (start time {dt.datetime.now()})\n')
    print('Progress\n--------')
    fin_hc = _sourcepath(modi,str1)
    fi=xr.open_dataset(fin_hc,decode_times=False)
    lon = fi.X.values # Lon
    lat = fi.Y.values # Lat
    time = fi.S.values # start time (months)
    nt = len(time)
    nl = len(fi.L.values) # lead time
    nm = len(fi.M.values) # member
    for il in (9,):
        sst=np.empty((nt,nm,len(lat),len(lon)))
        print(f'\nlead {il}\n   member')
        fout = f'{dirout}/{str1}/sst_{modi}_{str1}_l{il}.nc'
        # Loop through members
        for im in range(0,nm):
            # Subset times
            tstart = 0
            for tt in range(0,np.ceil(nt/time_chunk).astype(int)):
                tcount = min(time_chunk,nt-tstart+1)
                isdone = False
                while not isdone: # Loop to ensure download succeeds
                    try: 
                        sst[tstart:tstart+tcount,im,:,:]=fi.sst.isel(S=slice(tstart,(tstart+tcount)),
                                                                     M=im,L=il).values
                        tstart = tstart + tcount
                        isdone = True
                    except:
                        raise
                        print(' x')                
            print('   ',im)
        xout=xr.Dataset(data_vars={'lon':(['X',],lon),
                                'lat':(['Y',],lat),
                                'time':(['S'],time),
                                'sst':(['S','M','Y','X'],sst)},
                    coords=dict(X=fi.X,Y=fi.Y,M=fi.M,S=fi.S),
                    attrs={'description':'downloaded from http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME'})
        xout.to_netcdf(fout,mode='w')
        del sst; del xout; fi.close()
    return
for modi in mods:
    dl(modi,'hindcast')
    if modi in {'CanCM4i', 'GEM-NEMO','GFDL-SPEAR', 'NASA-GEOSS2S'}:
        dl(modi,'forecast')
print('\nDONE\n\n')



Progress
--------

lead 9
   member
    0
    1
    2
    3
    4
    5
    6
    7
    8
    9
    10
    11
    12
    13
    14
    15
    16
    17
    18
    19
    20
    21
    22
    23
    24
    25
    26
    27

DONE


